In [ ]:
# %%
import datetime
from csbdeep.io import load_training_data
from csbdeep.utils import axes_dict
from projection_upsampling_network.projection_upsampling_class import ProjectionUpsamplingConfig, ProjectionUpsampling
import tensorflow as tf
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr, mean_squared_error as mse, structural_similarity as ssim, normalized_root_mse as nrmse_sklearn
import csv
import datetime
from csbdeep.utils import normalize
import mrcfile
from skimage.transform import resize
import os
import tifffile as tiff

2025-04-21 21:33:51.679668: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-21 21:33:52.118301: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-21 21:33:53.876719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/student/a/asarker/.conda/envs/thesis/lib/:/home/student/a/asarker/.conda/envs/thesis/lib/python3.10/site-packages/nvidia/cudnn/lib:/home/student/a/asarker/.conda/envs/thesis/lib/:/home/student/a/asarker/.co